# UTD dataloader sanity test

In [ ]:
from pathlib import Path
import numpy as np
import random
import matplotlib.pyplot as plt
import yaml

DATA_DIR = (Path.cwd() / '../datasets/utd').resolve()
CONFIG_PATH = (Path.cwd() / '../conf/data/utd.yaml').resolve()
print('Data dir:', DATA_DIR)
assert DATA_DIR.exists(), 'Missing UTD dataset directory'
with CONFIG_PATH.open('r') as f:
    cfg = yaml.safe_load(f) or {}
POSE_FILE = cfg.get('pose_file', 'skeleton_aligned.npy')
ACC_FILE = cfg.get('acc_file', 'inertial_std.npy')
print('Using pose file:', POSE_FILE)
print('Using acc file:', ACC_FILE)

In [ ]:
window_size = 100
random.seed(0)
subjects = sorted([d for d in DATA_DIR.iterdir() if d.is_dir() and d.name.startswith('s')])
num_subjects = min(3, len(subjects))
for subject_dir in random.sample(subjects, k=num_subjects):
    pose_files = sorted(subject_dir.glob(f'a*_{subject_dir.name}_t*_{POSE_FILE}'))
    if not pose_files:
        continue
    pose_path = random.choice(pose_files)
    parts = pose_path.name.split('_')
    action, sub, trial = parts[0], parts[1], parts[2]
    pose = np.load(pose_path)
    acc_path = subject_dir / f"{action}_{subject_dir.name}_{trial}_{ACC_FILE}"
    if not acc_path.exists():
        continue
    acc = np.load(acc_path)
    length = min(pose.shape[-1], acc.shape[-1])
    max_start = max(0, length - window_size)
    start = random.randint(0, max_start)
    end = start + window_size
    pose_window = pose[:, :, start:end]
    acc_window = acc[:, start:end]
    fig, axs = plt.subplots(3, 2, figsize=(12, 6))
    fig.suptitle(f"{pose_path.name} window {start}:{end}")
    axes_labels = ['x', 'y', 'z']
    for axis in range(3):
        axs[axis, 0].plot(acc_window[axis], color='tab:blue', label='Acc')
        axs[axis, 0].set_ylabel(f'Acc {axes_labels[axis]}')
        axs[axis, 0].grid(True, alpha=0.3)
        axs[axis, 0].legend(loc='upper right')
        axs[axis, 1].plot(pose_window[axis, 8, :], color='tab:orange', label='Pose (joint 8)')
        axs[axis, 1].set_ylabel(f'Pose {axes_labels[axis]}')
        axs[axis, 1].grid(True, alpha=0.3)
        axs[axis, 1].legend(loc='upper right')
    axs[-1, 0].set_xlabel('timestep')
    axs[-1, 1].set_xlabel('timestep')
    plt.tight_layout(rect=[0, 0, 1, 0.94])
    plt.show()